# Testing Balanced Autoencoder on Old TL model 
- Weighted Accuracy: 0.641255253
- Sensitivity/Recall: 0.601520926
- Ephochs 100,300

In [ ]:
import os
import sys
import importlib
import pickle
import keras
import pdb
import pandas as pd
import numpy as np
from datetime import date, datetime

In [ ]:
execution_path = "../../"
import os
data_ndarr = np.load(r"../Data/all_pids_q=0.3.npy", allow_pickle=True)
datadict = data_ndarr.item()

In [ ]:
from TL_Packages import *

# Architecture_GROUP = "TL"
Architecture_GROUP = "model1"

def runtime():
    print("Runtime:", datetime.now().hour, "o' clock and", datetime.now().minute, "minutes")

def run_tlsi(file):
    """
        run transfer learning algorithm on sample imbalanced data set
    """
    pass

In [ ]:
p_ids = datadict.keys()
smaple_size_dict={}
print("patients: n =", len(p_ids), end='\n\n')

for p_id in p_ids:
    df = datadict[p_id]
    df.drop(['id','date','am_pef_org','BMI','sex', 'age'], axis=1, inplace=True) 
    df = np.asarray(df).astype(np.float32)
    print(p_id, "shape:", df.shape)#, end="\t\t")
    smaple_size_dict[p_id]=df.shape[0]

datadict[p_id].head(4)

#### dropped columns

`date` was dropped above, because it would throw `ValueError: could not convert string to float: '2017-11-01'`
`id` and `am_pef_org` were dropped because neither are explanatory variables.

(Note: `am_pef_org` is the UNSHIFTED, NON-NORMALIZED value for a person's morning exposure. It is relatively raw, and should almost exclusively be used for classification purposes)

In [ ]:
target_grp = ["SB-001", "SB-003", "SB-008", "SB-011", "SB-012", "SB-022", "SB-023", "SB-025", "SB-028", "SB-037", "SB-043", "SB-056", "SB-059", "SB-060", "SB-070", "SB-071", "SB-073", "SB-078", "SB-079", "SB-080", "SB-082", "SB-083", "SB-112"]

print(len(target_grp))

In [ ]:
from TL_Packages import *

# Architecture_GROUP = "TL"
Architecture_GROUP = "model"
RESULT_DIR = 'Results/'
EVALUATION_METRICS = ["Weighted Accuracy", "Sensitivity/Recall", "Specificity", "Precision_class0", "Precision_class1", "Precision_avg", "F1_class0", "F1_class1", "F1_avg", "auc_roc_score", "False_Discovery_Rate", "False_Negative_Rate", "False_Omission_Rate", "False_Positive_Rate", "Jaccard"]

# TL_Base_Model

In [ ]:
from tqdm import tqdm
import os.path

def TL_Base_Model(POPULATION_EPOCH_SIZE,PATIENT_EPOCH_SIZE, p_ids):
    df_evaluation_results = pd.DataFrame(columns=EVALUATION_METRICS)

    minority_variable=0
    #Target MOdel dataframe
    df_evaluation_results = pd.DataFrame(columns=EVALUATION_METRICS)
    #Population model results
    minor_var=minority_variable

    for target in tqdm(p_ids):  
        if target not in target_grp:
            continue
        runtime()
        print("~~~~~~~~~~~", POPULATION_EPOCH_SIZE, "~~~~~~~~~~~~~~~~~~~", PATIENT_EPOCH_SIZE, "~~~~~~~~~~~~~~~~~~~", target, "~~~~~~~~~~~~~~~~~~~")

        patient_dir = RESULT_DIR + "/diagrams/target"

        # create directory for this patient
        if not (patient_dir and patient_dir):
            os.makedirs(patient_dir)

        CLASS_VAR = 'class'
        # load population data
        X_train, X_test, Y_train, Y_test = split_data(target=target, data=datadict,  class_var=CLASS_VAR, minority_var=minor_var,test_frac=0.05,
                                                      transfer=True, balance=True)

        # pre-train model with population data
        net = network_tl(X_train, Y_train, n_hidden=32, learning_rate=0.001, device='cpu')
        net.make_pop()

        net.train(n_iters=POPULATION_EPOCH_SIZE)

        VERSION_TAG = f'TLSI_{Architecture_GROUP}_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}'
        #save model
        modelname = f"{target}-{VERSION_TAG}.hdf5"
        model_path= patient_dir + "/" + modelname
        net.nn_pop.save(model_path) #nn_pop

        #save training curve
        training_curve_file = patient_dir +"/"+ f'{VERSION_TAG}_training_curve_{target}'
        net.save_training_curve(training_curve_file)
        # load target patient data
        dataset_list = split_data(target=target, data=datadict,  class_var=CLASS_VAR, minority_var=minor_var, #explanatory_var
                                balance=True)

        kfold_evaluation_results_list = []
        kfold_confusion_matrix_list = []

        for i in range(3):
            # load the target data in the current round
            X_train, X_test, Y_train, Y_test = dataset_list[i]
            net.make_ind()
            net.retrain(X_train, Y_train, n_iters=PATIENT_EPOCH_SIZE)
            print("done RE-training", i, "of 3", end=". ")

            # test
            VERSION_TAG = f'TLSI_{Architecture_GROUP}_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}_{i}'
            #save model
            modelname = f"{target}-{VERSION_TAG}.hdf5"
            model_path= patient_dir + "/" + modelname
            net.nn_ind.save(model_path) #nn_pop

            #save retraining curve
            training_curve_file = patient_dir +"/"+ f'{VERSION_TAG}_training_curve_{target}'
            net.save_training_curve(training_curve_file)

            results, c_matrix = net.test(X_test, Y_test)
            kfold_evaluation_results_list += [results]
            kfold_confusion_matrix_list += [c_matrix]

            print("done testing", i, "of 3.")

        VERSION_TAG = f'TLSI_{Architecture_GROUP}_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}'

        # save average of evaluation results
        confusion_matrixes_file = (f"{str(patient_dir)}/{VERSION_TAG}_confusion_matrix_{target}")

        avg_results = pd.DataFrame(kfold_evaluation_results_list, columns=EVALUATION_METRICS).mean().values

        avg_results = avg_results.reshape(1, 15)

        df = pd.DataFrame(avg_results, columns=EVALUATION_METRICS)
        df.index = [f"{target}_{POPULATION_EPOCH_SIZE}_{PATIENT_EPOCH_SIZE}"]
        df_evaluation_results = pd.concat([df_evaluation_results,df], axis = 0)

        net.save_confusion_matrix(kfold_confusion_matrix_list, confusion_matrixes_file)

    # output_path = os.path.join(RESULT_DIR, f'TLSI_{Architecture_GROUP}_evaluation_metrics_{sorted(set(p_ids).difference(target_grp))[0]}.csv')
    output_path = os.path.join(RESULT_DIR, f'TLSI_{Architecture_GROUP}_evaluation_metrics.csv')

    if os.path.isfile(output_path):
        df_evaluation_results.to_csv(output_path,
                                        mode='a',
                                        header=False
                                    )
    else:
                df_evaluation_results.to_csv(output_path,
                                        mode='a'
                                    )

    print("evaluation results saved for", VERSION_TAG)

In [ ]:
POPULATION_EPOCH_SIZE = [*range(500, 1600, 500)]
PATIENT_EPOCH_SIZE = [*range(500, 1600, 500)]

print("POPULATION_EPOCH_SIZE:",POPULATION_EPOCH_SIZE, "Len:", len(POPULATION_EPOCH_SIZE))
print("PATIENT_EPOCH_SIZE:",PATIENT_EPOCH_SIZE, "Len:", len(PATIENT_EPOCH_SIZE))
print("target_grp:",target_grp, "Len:", len(target_grp))
print("Total records in each file:",len(POPULATION_EPOCH_SIZE) * len(PATIENT_EPOCH_SIZE) * len(target_grp))

# Execution

In [ ]:
TL_Base_Model(100,300, p_ids)

In [ ]:
print("Done")

# End